## Breast Cancer Dataset


Invasive Ductal Carcinoma (IDC) is the most common subtype of all breast cancers. To assign an aggressiveness grade to a whole mount sample, pathologists typically focus on the regions which contain the IDC. As a result, one of the common pre-processing steps for automatic aggressiveness grading is to delineate the exact regions of IDC inside of a whole mount slide.


The original dataset consisted of 162 whole mount slide images of Breast Cancer (BCa) specimens scanned at 40x. From that, 277,524 patches of size 50 x 50 were extracted.


The original data was collected from 279 patients. But we have modified this number to 1000 different patients i.e. the images have been distributed randomly across the 1000 unique patient ids. This has been to done to demonstrate that through our library PySyft we can perform multiple queries while preserving the privacy of a large group of people at scale.

Further we have created 100 unique subsets from the dataset and distributed across the participants so that each one can act as an individual Data Owner.

In [1]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import shutil
import uuid
from PIL import Image

In [2]:
def calc_label_frequency(labels):
    from collections import defaultdict
    labels = labels.flatten() if hasattr(labels, "flatten") else labels
    freq = defaultdict(int)
    for label in labels:
        freq[label] += 1
    return dict(freq)

In [3]:
def check_data_distribution(data_df):
    data_distribution = {}
    for patient_id in data_df["patient_ids"].unique():
        temp = data_df[data_df["patient_ids"] == patient_id]
        data_distribution[patient_id] = temp.shape[0]
    print(data_distribution)
    print(min(data_distribution.values()), max(data_distribution.values()))

In [4]:
def generate_random_ids(num=1000):
    patient_ids = set()
    while(len(patient_ids) < num):
        patient_id = np.random.randint(1000, 1000000)
        patient_ids.add(patient_id)
    return patient_ids

In [5]:
def read_data_from_disk(data_path = "archive/"):
    data_list_dict = []
    for patient_dir in tqdm(os.listdir(data_path)):
        if patient_dir == "IDC_regular_ps50_idx5":
            continue

        patient_id = int(patient_dir)
        patient_dir = data_path + patient_dir + "/"
        label_dirs = os.listdir(patient_dir)
        for label in label_dirs:
            image_file_path = patient_dir + label + "/"
            image_files = os.listdir(image_file_path)
            for image_name in tqdm(image_files):            
                data_list_dict.append(
                    {
                        "patient_ids": patient_id,
                        "labels": label,
                        "image_paths": image_file_path + image_name,
                    }
                )
        data_df = pd.DataFrame(data_list_dict)
    return data_df

In [6]:
# Create data subset directory

data_subset_folder = "BreastCancerDataset/subsets"
if os.path.exists(data_subset_folder):    
    print("Data subset directory already Exists. Clearing existing one.")
    shutil.rmtree(data_subset_folder)

os.makedirs(data_subset_folder)
print("Data subset directory created.")

Data subset directory created.


In [7]:
TOTAL_PARTICIPANTS = 100

def create_data_subsets(data_df, patient_ids):
    data_subsets_map = {}
    data_df.sort_values("patient_ids", inplace=True, ignore_index=True)
    
    start = 0
    for participation_number in tqdm(range(1, TOTAL_PARTICIPANTS+1)):

        # Calculate start and end index based on your participant number
        batch_size = data_df.shape[0] // TOTAL_PARTICIPANTS
        start_idx = (participation_number - 1) * batch_size
        end_idx = start_idx + batch_size

        # Slice the dataframe according
        subset = data_df[start_idx:end_idx]

        # Reset index of the subset
        subset.reset_index(inplace=True, drop=True)
        
        patient_id_list = [np.random.choice(patient_ids[start: start+10]) for i in range(subset.shape[0])]
        subset["patient_ids"] = patient_id_list
        
        print("Reading Images as array.....")
        
        images_as_array = []
        for image_filepath in subset["image_paths"]:
            img = np.asarray(Image.open(image_filepath))
            images_as_array.append(img)
            
        del subset["image_paths"]
        subset["images"] = images_as_array
        
        print("Done storing Images as array.")
        
        subset_filename = f"BreastCancerDataset-{uuid.uuid4().hex[:TOTAL_PARTICIPANTS]}.pkl"
        subset_path = f"{data_subset_folder}/{subset_filename}"
        subset.to_pickle(subset_path)
        data_subsets_map[participation_number] = subset_filename
        
    return data_subsets_map
    print("Data subsets Created Successfully !!!")

In [9]:
data_df = read_data_from_disk()
random_patient_ids = generate_random_ids(1000)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1636/1636 [00:00<00:00, 685776.67it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [00:00<00:00, 178388.45it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 531861.98it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 143/143 [00:00<00:00, 940102.62it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 597/597 [00:00<00:00, 2171725.49it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 110/110 [00:00<00:00, 117

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1066/1066 [00:00<00:00, 2664557.85it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 650/650 [00:00<00:00, 2848795.82it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 714/714 [00:00<00:00, 1244693.71it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 272/272 [00:00<00:00, 968464.08it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1802/1802 [00:00<00:00, 2379016.62it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 53

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 377/377 [00:00<00:00, 1971636.67it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 905245.47it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 1913459.85it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [00:00<00:00, 2454041.72it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:00<00:00, 2650149.96it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 504/504 [00:00<00:00, 301

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 450/450 [00:00<00:00, 2684831.86it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 420/420 [00:00<00:00, 1961701.20it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 190/190 [00:00<00:00, 1972568.71it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 375/375 [00:00<00:00, 2187571.63it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 542/542 [00:00<00:00, 2661958.74it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 842/842 [00:00<00:00, 223

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46/46 [00:00<00:00, 1261032.58it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1174/1174 [00:00<00:00, 2636034.74it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 225/225 [00:00<00:00, 2681018.18it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 927/927 [00:00<00:00, 2659452.67it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 264/264 [00:00<00:00, 2499540.08it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 469/469 [00:00<00:00, 216

...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 240/240 [00:00<00:00, 2164802.06it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 287/287 [00:00<00:00, 2859299.88it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 554/554 [00:00<00:00, 2219335.64it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 427/427 [00:00<00:00, 2010064.88it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 208/208 [00:00<00:00, 2403347.75it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 581/581 [00:00<00:00, 238

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 837/837 [00:00<00:00, 2527453.17it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 978/978 [00:00<00:00, 2583141.88it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [00:00<00:00, 1442994.50it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1485/1485 [00:00<00:00, 2697506.04it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 168/168 [00:00<00:00, 2090929.00it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 198/198 [00:00<00:00, 187

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1001/1001 [00:00<00:00, 2459577.21it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 201/201 [00:00<00:00, 2764115.10it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 283/283 [00:00<00:00, 1908340.89it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1913/1913 [00:00<00:00, 2646340.22it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 699050.67it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 435/435 [00:00<00:00, 223

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 656/656 [00:00<00:00, 2483270.24it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 37/37 [00:00<00:00, 891892.23it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 687/687 [00:00<00:00, 2161655.55it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 426/426 [00:00<00:00, 2171049.22it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1421/1421 [00:00<00:00, 2547053.84it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 590/590 [00:00<00:00, 216

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 862/862 [00:00<00:00, 2416771.42it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 394/394 [00:00<00:00, 1892961.94it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 105/105 [00:00<00:00, 1287724.91it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1066/1066 [00:00<00:00, 2478452.36it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 624/624 [00:00<00:00, 2143526.37it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 280/280 [00:16<00:00

In [10]:
data_df.head()

,patient_ids,labels,image_paths
0,9265,0,archive/9265/0/9265_idx5_x2551_y601_class0.png
1,9265,0,archive/9265/0/9265_idx5_x2051_y601_class0.png
2,9265,0,archive/9265/0/9265_idx5_x2551_y451_class0.png
3,9265,0,archive/9265/0/9265_idx5_x1851_y2251_class0.png
4,9265,0,archive/9265/0/9265_idx5_x1501_y2251_class0.png


In [11]:
calc_label_frequency(data_df.labels)

{'0': 198738, '1': 78786}

In [12]:
len(random_patient_ids)

1000

In [13]:
data_subset_map = create_data_subsets(data_df, list(random_patient_ids))

  0%|                                                                                                                                          | 0/100 [00:00<?, ?it/s]/tmp/ipykernel_117590/1688677773.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset["patient_ids"] = patient_id_list


Reading Images as array.....


/tmp/ipykernel_117590/1688677773.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset["images"] = images_as_array
  1%|█▎                                                                                                                                | 1/100 [00:00<01:30,  1.09it/s]/tmp/ipykernel_117590/1688677773.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset["patient_ids"] = patient_id_list


Done storing Images as array.
Reading Images as array.....


/tmp/ipykernel_117590/1688677773.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset["images"] = images_as_array
  2%|██▌                                                                                                                               | 2/100 [00:01<01:20,  1.22it/s]/tmp/ipykernel_117590/1688677773.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset["patient_ids"] = patient_id_list


...

Done storing Images as array.
Reading Images as array.....


/tmp/ipykernel_117590/1688677773.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset["images"] = images_as_array
 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋ | 99/100 [01:21<00:00,  1.16it/s]/tmp/ipykernel_117590/1688677773.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset["patient_ids"] = patient_id_list


Done storing Images as array.
Reading Images as array.....


/tmp/ipykernel_117590/1688677773.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset["images"] = images_as_array
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:21<00:00,  1.22it/s]

Done storing Images as array.


In [14]:
data_subset_map

{1: 'BreastCancerDataset-02ec48b840824b1ea3e1f5d11c45314b.pkl',
 2: 'BreastCancerDataset-1cae7adbf40448d0b488f98922dd3c19.pkl',
 3: 'BreastCancerDataset-b78c985152284c53b4c3a94f7e047836.pkl',
 4: 'BreastCancerDataset-229bbe185f4d40acb52ca52c9cf27fd7.pkl',
 5: 'BreastCancerDataset-940253bbc09643eda8fa2af6c45888a5.pkl',
 6: 'BreastCancerDataset-1eb7c5645e3a4240b0e9d5cc765404cf.pkl',
 7: 'BreastCancerDataset-3e103666e1ce485080cb2313cd51f6ee.pkl',
 8: 'BreastCancerDataset-1acfdc2b6e52494eb5cd834610e94a6a.pkl',
 9: 'BreastCancerDataset-c71d75a6650043ffa23430ef9436d0d3.pkl',
 10: 'BreastCancerDataset-ae3e513b3ddc402ba46e74cee09ba56a.pkl',
 11: 'BreastCancerDataset-e0c3567f756e4f97b9ade3b7275eda73.pkl',
 12: 'BreastCancerDataset-71b4ca3f4a094777a1b8f39013efb690.pkl',
 13: 'BreastCancerDataset-04e26e591a724662af3b9a7c84e62d92.pkl',
 14: 'BreastCancerDataset-2082593da1174737bf1368b36e4e6070.pkl',
 15: 'BreastCancerDataset-c01c514137ba4004804f063dbd0089a9.pkl',
 16: 'BreastCancerDataset-e3fc7eb4

In [15]:
with open("BreastCancerDataset/dataset.json", "w") as fp:
    json.dump(data_subset_map, fp)

In [16]:
assert len(os.listdir(data_subset_folder)) == TOTAL_PARTICIPANTS, "Subsets are less than TOTAL PARTICIPANTS"

Upload `dataset.json` and each of the data subsets to `https://github.com/OpenMined/datasets/tree/main/BreastCancerDataset`.